# Analyzing Grouped or Clustered Data with Multilevel Models

We encounter clustered data all the time: survey respondents clustered by city or county, traffic data clustered by ward, developments clustered by zoning regulation, reports clustered by agency, tests clustered by student or patient, and test scores clustered by school and district. Most of the data we use, in fact, could be conceptuatlized as being derived from a *multilevel data generating process (DGP)*. OLS assumes our observations are independent of one another and result in errors that are *i.i.d.* Clustered data violate such assumptions. So how to model clustered data in R?

For this activity, we will examine precinct-level results from OH's 2023 [Proposition 1 which codified the right to an abortion](https://ballotpedia.org/Ohio_Issue_1,_Right_to_Make_Reproductive_Decisions_Including_Abortion_Initiative_(2023)) into the state's constitution. We will rely on two data sources: The Ohio Secretary of State's [Election Data and Results](https://www.ohiosos.gov/elections/election-results-and-data/) page (for election results) and the county-level demographic data from the 2020 decennial census from the U.S. Census Bureau. 

For this activity, please complete all code chunks below in Quarto and render as a pdf for submission. You can skip the `install.packages()` code chunk in Quarto (but you might need to install a package if you have not previously done so). Be sure to complete any requested tasks as you work through the tutorial.

Install any needed packages. We'll be using `lme4` and `estimatr`, which might be new to you. `lme4` will be used to run RE models, while `estimatr` offers a simple interface for specifying FE models and clustering standard errors. 

In [ ]:
# install any of the packages below that you need

# install.packages('tidyverse')
# install.packages('lme4')
# install.packages('estimatr')
# install.packages('marginaleffects')
# install.packages('modelsummary')

In [ ]:
for(pack in c('tidyverse', 'lme4', 'estimatr', 'marginaleffects','modelsummary')){
    library(pack, character.only = TRUE)
}

In [ ]:
# get prop 1 election data and create outcome measured as proportion
oh <- read.csv("https://raw.githubusercontent.com/bowendc/512_labs/refs/heads/main/oh_2023.csv") |>
            mutate(yes.pct = 100 * Prop1.Yes / (Prop1.Yes + Prop1.No))

# get oh gov election datat from 2022 and create % variable
ohgov <- read.csv("https://raw.githubusercontent.com/bowendc/512_labs/refs/heads/main/oh_gov_2022.csv") |>
            mutate(demgov.pct  = 100 * dem22gov / (dem22gov + rep22gov),
                   cnt.demgov = demgov.pct - mean(demgov.pct, na.rm = TRUE))

# get census data 
county <- read.csv("https://raw.githubusercontent.com/bowendc/512_labs/refs/heads/main/oh_county.csv")

Examine each dataframe. You can use `View(df)` to look at the raw data. You do not need to include those functions in your Quarto document. Using a Markdown chunk, describe the unit of analysis for each dataframe.  

Clearly, we need to process `county` to get it down to the county level. We can use `pivot_wider` from the `dplyr` package (part of the `tidyerse`) to transpose the dataframe. We just need to describe where the data values come from and where the names come from. Technically, we're moving from a "long" dataframe to a "wide" dataframe. That is, we're going to express information once stored in **rows** and move them into new **columns**. 

In [ ]:
county.wide <- county |>
            pivot_wider(id_cols = c(GEOID, NAME), # identifies the unit
                names_from = variable,   # where the new var names come from
                values_from = value)     # source of data values


Let's check to make sure our transposition worked the way we want.

In [ ]:
head(county.wide)

Ok, now we have county data that we can combine with our precinct data -> two separate levels of data where one is nested within the other. For our own ease of analysis, we can combine these dataframes together. Let's check our other two dataframes again and identify the county variables.

In [ ]:
head(oh, n = 3)
head(ohgov, n = 3)

Uh oh. Looks like in `oh` and `ohgov` (our precinct data), the county is stored in `County.Name` variable and includes only the name. In `county_wide`, the variable `NAME` includes the name followed by "County, Ohio". We need to either remove "County, Ohio" from `county_wide` or add it to the other two dataframes. Fortunately, we can accomplish this using string functions that allow us to manipulate text data.

Let's add a new variable, titled the same as the county variable is titled in the `oh` and `ohgov` dataframes for convenience later. This new variable will remove the unnecessary text behind the county name. We are fortunate because each county name has the same character string behind it, so it will be easy to remove.

In [ ]:
# gsub function looks for a pattern (our text we want changed)
# and then replaces it with a new pattern (here I use "" with
# no space between the quotation marks. That will replace 
# the pattern with no other text.). The last arg specifies 
# the vector of data. Here, it is the variable `NAME`. 

county.wide <- county.wide |> 
                mutate(County.Name = gsub(" County, Ohio", "", NAME))

head(county.wide$County.Name, n = 3)

Now we're ready to combine dataframes. We can use functions from `tidyverse`'s `dpylr` package.

In [ ]:
# left_join will combine two dataframes based on the variables listed in
# by(). Here we use the county name and the precinct code to do our matches
# Use codes instead of names when possible. 

# left_join will keep only unmatched observations from the first df you list
# if the observation is unmatched but included in the second df, it will
# be dropped from the combined df.
oh.comb <- left_join(oh, ohgov, by = c("County.Name", "Precinct.Code"))

nrow(oh)
nrow(ohgov)
nrow(oh.comb)

# it is ALWAYS a good idea to inspect your dataframe after merging!
head(oh.comb)

# let's run left_join again and merge in county data
oh.comb <- left_join(oh.comb, county.wide, by = "County.Name")

nrow(oh.comb)
head(oh.comb)


Notice that every precinct in a county recevies the same values from the `county.wide` dataframe - they are represented in the last 5 variables in the combined dataframe. 

Ok, now we are almost ready to analyze our multilevel data. Let's create a new variable to record if an abortion provider facility is located in the county. In 2023, facilities providing abortions existed in six OH counties. 

In [ ]:
# create list storing the names of counties with abortion facilities
countylist <- c("Cuyahoga", "Hamilton", "Franklin", 
           "Summit", "Montgomery", "Lucas")

# the ifelse() statement below will be coded 1 if an observation's 
# County.Name is found in list `countylist` and 0 otherwise
oh.comb <- oh.comb |> 
            mutate(access = ifelse(County.Name %in% countylist, 1, 0))

## Clustering Standard Errors

The simplest way to address some of the issues of clustered data is to adjust coefficient standard errors to allow for cluster-correlated error terms. Of course, this approach handles our lack of independent observations but not address un-modeled group heterogeneity.

First, let's fit a plain old OLS model predicting support for Prop 1 by the percent of the precinct voting for the Democratic gubernatorial candidate in the previous election. Then, we'll use `lm_robust()` from the `estimatr` package to estimate the OLS model with group-clustered standard errors.

In [ ]:
m.ols <- lm(yes.pct ~ demgov.pct  + log(totpop) + access +
                        pct.black + pct.latino, 
                        data = oh.comb)

m.clse <- lm_robust(yes.pct ~ demgov.pct + log(totpop) + access +
                        pct.black + pct.latino, 
                        cluster = County.Name,     # tells R to cluster SE by county var
                        se_type = 'stata',         # standard formula for clustering
                        data = oh.comb)

# this is a nice way of formating a regression table. It will set the names 
# we want the coefficients to have in our table. Store as a list.

coefmap <- c("(Intercept)" = "Constant",
            "demgov.pct" = "Precinct Vote for Dem Gov",
            "log(totpop)" = "County Population (logged)",
            "access" = "Abortion Facility in County",  
            "pct.black" = "Proportion Black in County",
            "pct.latino" = "Proportion Latino/a in County")

# use modelsummary to rename models 
modelsummary(list("OLS" = m.ols, "OLS with Clustered SEs" = m.clse),
    stars = TRUE,
    estimate = "{estimate}{stars}",
    statistic = "({std.error})",
    coef_map = coefmap,                       # calls up our stored list
    gof_map = c("nobs", "r.squared", "rmse")) # identifies just our needed stats

Using Markdown, describe the differences between the regular OLS estimates and the OLS estimates with clustered standard errors by county. What types of estimates are impacted, and what difference does the clustering make for how we interpret the regression results?

## Fixed Effects (FE)

Fixed effects models take a different approach. Instead of relaxing the assumption of independent errors, FE models try to model the inter-relatedness of our observations by fitting a dummy variable (or demeaning the data) for each group. So FEs "soak up" the variation in $y$ coming from the group-level. 

In [ ]:
# here, we use lm_robust. First, create FE by including County.Name
# this works because it is a factor variable. R will create
# dummy vars for each county and include in model

m.fe <- lm_robust(yes.pct ~ demgov.pct + log(totpop) + 
                        access + pct.black + pct.latino + factor(County.Name),
                        data = oh.comb)

# you can see the dummies if you run: 
summary(m.fe)


# if we use the `fixed_effects` estimator directly, it will 
# "demean" the data: meaning, it will subtract the mean value 
# of y for the group, removing any county-level average 
# differences. The only remaining variation is coming from within
# the counties (our precincts), not across them
m.fe.within.panel.est <- m.fe.clse <- lm_robust(yes.pct ~ demgov.pct + log(totpop) + 
                        access + pct.black + pct.latino,
                        fixed_effects = ~County.Name,
                        data = oh.comb)

# we could also still cluster the SEs, but this is often not
# necessary
m.fe.clse <- lm_robust(yes.pct ~ demgov.pct + log(totpop) + 
                        access + pct.black + pct.latino,
                        fixed_effects = ~County.Name, 
                        cluster = County.Name,
                        se_type = 'stata',
                        data = oh.comb)

modelsummary(list("FE" = m.fe, 
                  "FE within-panel estimator" = m.fe.within.panel.est, 
                  "FE with Clustered SEs" = m.fe.clse),
                stars = TRUE,
                estimate = "{estimate}{stars}",
                statistic = "({std.error})",
                coef_map = coefmap,
                gof_map = c("nobs", "r.squared", "rmse"))

Hmm... where did our county-level variables go? The first model (with our included dummy variables) drops `access`, while all county-level variables are droped in the next two models. Say why you think this happened in a Markdown section.

## Random Effects (REs)

So-called random effects models take a different approach, separating the variation in $y$ into "fixed" portion and "random" portions which are assumed to be independent of one another. The random portion has an assumed distribution (usually normal) and can be used to describe the unmodeled group-heterogeneity. Both group-level intercepts and slopes can be included as random effects, and we can create point estimates by group from the model results. This method does let us include group-level predictors. Below, we fit random effects models using `lmer` from the `lme4` package. There is a corresponding `glmer` function for generalized linear models as well.

In [ ]:
# fit a random intercept model with no other predictors 
# besides the intercept (hence the 1)
m.re <- lmer(yes.pct ~ 1 + 
        (1 | County.Name),  # this means random intercept for each level of county
        data = oh.comb)

# fit full random intercept model including "fixed" portion (standard coefficients)
m.re.ri <- lmer(yes.pct ~ demgov.pct + log(totpop) + 
                        access + pct.black + pct.latino +
                        (1 | County.Name),
                        data = oh.comb)

# fit random intercept, random slope model. 
m.re.rirs <- lmer(yes.pct ~ demgov.pct + log(totpop) + 
                        access + pct.black + pct.latino +
                        (1 + demgov.pct| County.Name), # now, we have intercept and 
                        data = oh.comb)                # slopes for demgov.pct that 
                                                       # vary by county

modelsummary(list("Naive Random Intercept" = m.re, 
                  "Random Intercept" = m.re.ri, 
                  "Random Intercepts and Slope" = m.re.rirs), 
                stars = TRUE,
                estimate = "{estimate}{stars}",
                statistic = "({std.error})",
                gof_map = c("nobs", "icc", "rmse", "aic", "bic"))

Notice that we can now get estimates for all county-level variables. Notice also that the coefficient estimate for `demgov.pct` remains very similar to that from the FE models until we let the slope vary by county (final model above). In that model, the average coef is quite a bit larger, although some counties will have lower slopes and some higher. 

The ICC (intra-class correlation coefficient) shows the proportion of the unexplained variance in $y$ coming from the group-level. In the naive model with no predictors, `lmer` estimates half of the variation in $y$ comes from the county level and half from the precinct level. When we re-estimate with our predictor variables included, we receive the same 0.5 ICC result. However, when we let the slopes of `demgov.pct` vary by county, we are better able to capture the precint-level variation in Prop 1 support. Now 80% of the remaining variation in $y$ comes from the county level.

It's a good idea to plot the results of multilevel model if you are interested in the group-level heterogeneity. Let's present both our random intercept and our random-intercept, random-slope models to predict the relationship between precinct-level Democratic voteshare and Prop 1 support. Of note, we need to let this relationship vary by county!

In [ ]:

# plot_predictions() is from the marginaleffects package

# first for just the random intercept model
plot_predictions(m.re.ri, condition = list("demgov.pct", "County.Name")) +
      theme_minimal() + theme(legend.position = "none")

# now the random intercept, random slope model
plot_predictions(m.re.rirs, condition = list("demgov.pct", "County.Name")) +
      theme_minimal() + theme(legend.position = "none")

This is ugly and hard to read. Let's store the predictions and plot using `ggplot()`

In [ ]:
out <- plot_predictions(m.re.rirs, condition = list("demgov.pct", "County.Name"),
                 draw = FALSE) 

# geom_rug gives those tick marks on the x-axis to show the distribution of values.
# sides = "b" will include only the rug plot on the x-axis, not the y

# note also that I am using the original data, not predicted data, for the rug plot
# but am using the predicted data (`out`) for the geom_line 

ggplot() +
    geom_line(data = out, aes(demgov.pct, estimate, group = County.Name),
        color = "black", alpha = .1) + 
    geom_rug(data = oh.comb, aes(x = demgov.pct), sides = "b", alpha = .025) + 
    theme_minimal() + theme(legend.position = "none")

Obviously, the effect of Democratic strength at the precinct level appears to vary by county, with the relationship much stronger in some counties than others. We could use this information to think more deeply about why - maybe we need to account for urbanism, education, relgiosity, or other factors if we want to better understand this differing relationship across places. 